# 第8章: 機械学習

本章では，Bo Pang氏とLillian Lee氏が公開している[Movie Review Data](http://www.cs.cornell.edu/people/pabo/movie-review-data/)の[sentence polarity dataset v1.0](http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.README.1.0.txt)を用い，文を肯定的（ポジティブ）もしくは否定的（ネガティブ）に分類するタスク（極性分析）に取り組む．

## 70. データの入手・整形
[文に関する極性分析の正解データ](http://www.cs.cornell.edu/people/pabo/movie-review-data/rt-polaritydata.tar.gz)を用い，以下の要領で正解データ（sentiment.txt）を作成せよ．

rt-polarity.posの各行の先頭に"+1 "という文字列を追加する（極性ラベル"+1"とスペースに続けて肯定的な文の内容が続く）
rt-polarity.negの各行の先頭に"-1 "という文字列を追加する（極性ラベル"-1"とスペースに続けて否定的な文の内容が続く）
上述1と2の内容を結合（concatenate）し，行をランダムに並び替える
sentiment.txtを作成したら，正例（肯定的な文）の数と負例（否定的な文）の数を確認せよ．

In [10]:
# done using shellscripts.
# add '+1' or '-1' by using vim.
# using VISUAL BLOCK mode, choose all rows, and then press shift + i, entering intert mode, then finally insety '+1' or '-1' and press esc.
# it insets '+1' or '01' to all rows.

#!cat rt-polarity.neg rt-polarity.pos | gshuf > sentiment.txt

!cat sentiment.txt | grep '^\+1' | wc -l
!cat sentiment.txt | grep '^\-1' | wc -l

    5331
    5331


## 71. ストップワード
英語のストップワードのリスト（ストップリスト）を適当に作成せよ．さらに，引数に与えられた単語（文字列）がストップリストに含まれている場合は真，それ以外は偽を返す関数を実装せよ．さらに，その関数に対するテストを記述せよ．

In [43]:
class stopWord:
    def __init__(self):
        self.words = 'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your'.lower().split(',')
        
    
    def exists(self, word):
        if word.lower() in self.words:
            return True
        else:
            return False
        

stopword = stopWord()


assert stopword.exists('a')
assert stopword.exists('after')
assert stopword.exists('among')
assert stopword.exists('are')

assert not stopword.exists('test')
assert not stopword.exists('check')
assert not stopword.exists('paper')


## 72. 素性抽出
極性分析に有用そうな素性を各自で設計し，学習データから素性を抽出せよ．素性としては，レビューからストップワードを除去し，各単語をステミング処理したものが最低限のベースラインとなるであろう．

In [105]:
import codecs
import snowballstemmer
import re
from collections import Counter

class stopWord_Stem:
    def __init__(self):
        self.words = 'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your'.lower().split(',')
        self.interjections = ['!', '?', 'ah', 'ahh', 'ahem', 'aww', 'aw', 'argh', 'bab', 'boo', 'duh', 'eek', 'eh', 'eep', 'gee', 'grr', 'hah', 'hmm', 'haha', 'huh', 'meh', 'ick', 'yuck', 'ich', 'yak', 'meh', 'mhm', 'mm', 'oh', 'iy', 'oy', 'pew', 'uh', 'uhh', 'wow']
        self.num = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
        self.other_word = ['de', 'niro', 'goe']
        self.stemmer = snowballstemmer.stemmer('english')
        self.stm_list = []
        self.features = []
        self.word_counter = Counter()

    def is_stopWord(self, word):
        if word.lower() in self.words:
            return True
        else:
            return False

    def is_interjections(self, word):
        if word.lower() in self.interjections:
            return True
        else:
            return False
        
    def is_num(self, word):
        if word.lower() in self.num:
            return True
        else:
            return False

    def is_someword(self, word):
        if word.lower() in self.other_word:
            return True
        else:
            return False

    def get_features(self, txt):
        
        for line in txt:
            for word in line[3:].split():
                
                word = word.strip()
                
                if re.search('[0-9(\'d)(\'v)\/\[\]\-]', word):
                    continue
                
                elif self.is_stopWord(word) or self.is_interjections(word) or self.is_num(word):
                    continue
                
                stm_word = self.stemmer.stemWord(word)
                
                if stm_word != '!' and stm_word != '?' and len(word) <= 2:
                    continue
                
                if self.is_someword(stm_word):
                    continue
        
    
    
                
                self.word_counter.update([stm_word])
        
        features = [word for word, count in self.word_counter.items() if count >= 6]
        
        return features
        
     
    
txt_file = 'sentiment.txt'
result_file = 'features.txt'

process = stopWord_Stem()

with codecs.open(txt_file, 'r', 'cp1252') as f:
    features = process.get_features(f)
    for i in features:
        
        print(i)

with codecs.open(result_file, 'w', 'cp1252') as f:
    for i in features:
        f.write(i)
        f.write('\n')

bite
terror
grim
gori
flourish
artist
onscreen
generat
spark
generous
honor
cheesi
spell
set
easi
pianist
highlight
thriller
hour
barri
tens
mental
cain
guilti
particular
wors
chaotic
tone
along
text
enthral
caught
premis
eleg
resort
suspect
remot
stale
inan
histor
piec
energet
taken
horror
listen
perhap
essenc
abstract
initi
raunchi
expos
sequel
impress
repres
punish
happen
polanski
pointless
mark
hero
belong
check
smith
matine
rate
term
greek
type
amateurish
character
enlighten
expert
machin
quick
japan
nichola
elabor
sober
bourn
minut
effort
likabl
written
shock
reflect
teach
close
kick
communiti
situat
tension
reach
simon
exot
brillianc
until
ritchi
shout
substanc
matur
hing
counterpart
irish
onto
patholog
replet
sweet
embrac
name
como
frank
bone
setup
ear
talki
yourself
menac
forth
testament
ryan
insomnia
mysteri
mike
cat
castro
grey
miracl
meant
sitcom
earli
cinemat
leap
frustrat
flimsi
protagonist
pena
nail
sublim
reason
work
aim
exquisit
interest
encourag
brian
bullock
teenag
f

## 73. 学習
72で抽出した素性を用いて，ロジスティック回帰モデルを学習せよ．

In [107]:
import codecs
import snowballstemmer
import numpy as np

    
class logit:
    def __init__(self, sentiments, features):
        
        self.words = 'a,able,about,across,after,all,almost,also,am,among,an,and,any,are,as,at,be,because,been,but,by,can,cannot,could,dear,did,do,does,either,else,ever,every,for,from,get,got,had,has,have,he,her,hers,him,his,how,however,i,if,in,into,is,it,its,just,least,let,like,likely,may,me,might,most,must,my,neither,no,nor,not,of,off,often,on,only,or,other,our,own,rather,said,say,says,she,should,since,so,some,than,that,the,their,them,then,there,these,they,this,tis,to,too,twas,us,wants,was,we,were,what,when,where,which,while,who,whom,why,will,with,would,yet,you,your'.lower().split(',')
        self.interjections = ['!', '?', 'ah', 'ahh', 'ahem', 'aww', 'aw', 'argh', 'bab', 'boo', 'duh', 'eek', 'eh', 'eep', 'gee', 'grr', 'hah', 'hmm', 'haha', 'huh', 'meh', 'ick', 'yuck', 'ich', 'yak', 'meh', 'mhm', 'mm', 'oh', 'iy', 'oy', 'pew', 'uh', 'uhh', 'wow']
        self.num = ['one', 'two', 'three', 'four', 'five', 'six', 'seven', 'eight', 'nine', 'ten']
        self.other_word = ['de', 'niro', 'goe']
        self.stemmer = snowballstemmer.stemmer('english')       
        
        self.features = []
        self.sentiments = []
        with open(features, 'r') as f:
            for word in f:
                self.features.append(word.strip())
                
        with open(sentiments, 'r') as f:
            for line in f:
                self.sentiments.append(line.strip())
        
        self.num_features = len(self.features)
        self.num_sentiments = len(self.sentiments)
        
        self.data_x = np.zeros([self.num_sentiments, self.num_features + 1], dtype=np.float64)
        self.data_y = np.zeros(self.num_sentiments, dtype=np.float64)
        
        np.random.seed(1)
        self.weights = np.random.randn(self.num_features, dtype=np.float64)
        
        for i, line in enumerate(sentiments):
            
            self.data_x[i] = self._extract_features(line[3:])
            
            if line[0:2] == '+1':
                self.data_y[i] = 1
        
    def is_stopWord(self, word):
        if word.lower() in self.words:
            return True
        else:
            return False

    def is_interjections(self, word):
        if word.lower() in self.interjections:
            return True
        else:
            return False
        
    def is_num(self, word):
        if word.lower() in self.num:
            return True
        else:
            return False

    def is_someword(self, word):
        if word.lower() in self.other_word:
            return True
        else:
            return False

    
    def _extract_features(self, txt):
        data = np.zeros(self.num_features + 1, dtype=np.float64)
        data[0] = 1
        
        for word in txt:
            word = word.strip()
            if self.is_stopWord(word) or self.is_interjections(word) or self.is_num(word) or self.is_someword(word):
                continue
        
            stm_word = self.stemmer.stemWord(word)
            
            if stm_word in self.features:
                data[self.features.index(stm_word)] = 1

        return data
                
        
    def loss(out ,y):
        
        #res = sigmoid(x.dot(weights))
        
        return 1
        
    def sigmoid(x):
        return 1.0 / (1.0 + np.exp(-x))
        

## 74. 予測
73で学習したロジスティック回帰モデルを用い，与えられた文の極性ラベル（正例なら"+1"，負例なら"-1"）と，その予測確率を計算するプログラムを実装せよ．

# 75. 素性の重み
73で学習したロジスティック回帰モデルの中で，重みの高い素性トップ10と，重みの低い素性トップ10を確認せよ．

## 76. ラベル付け
学習データに対してロジスティック回帰モデルを適用し，正解のラベル，予測されたラベル，予測確率をタブ区切り形式で出力せよ．

## 77. 正解率の計測
76の出力を受け取り，予測の正解率，正例に関する適合率，再現率，F1スコアを求めるプログラムを作成せよ．

## 78. 5分割交差検定
76-77の実験では，学習に用いた事例を評価にも用いたため，正当な評価とは言えない．すなわち，分類器が訓練事例を丸暗記する際の性能を評価しており，モデルの汎化性能を測定していない．そこで，5分割交差検定により，極性分類の正解率，適合率，再現率，F1スコアを求めよ．

## 79. 適合率-再現率グラフの描画
ロジスティック回帰モデルの分類の閾値を変化させることで，適合率-再現率グラフを描画せよ．